Segmenting and Clustering Neighborhoods in Toronto

Importing BeautifulSoup for WebScraping

In [30]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup

This is to read data from wiki in a Pandas dataframe

In [31]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

This is to transform data as per guidelines in the Assignment

In [32]:
neighborhoods = df[0]
neighborhoods.columns = ['PostalCode', 'Borough', 'Neighborhood']
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned'].reset_index(drop=True)
neighborhoods.loc[neighborhoods.Neighborhood == 'Not assigned', 'Neighborhood'] = neighborhoods['Borough']
neighborhoods['Neighborhood'] = neighborhoods.groupby(['PostalCode'])['Neighborhood'].transform(lambda x: ','.join(x))

neighborhoods = neighborhoods.drop_duplicates()
neighborhoods


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge,Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens,Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson,Garden District"


In [33]:
neighborhoods.shape

(103, 3)

In [35]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!pip install geocoder
import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geographiclib-1.49   | 32 KB     | ##################################### | 100% 
geopy-1.20.0         | 57 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages

In [66]:
ds = pd.read_csv("http://cocl.us/Geospatial_data")
ds.columns = ['PostalCode', 'Latitude', 'Longitude']

In [67]:
for postalcode in neighborhoods['PostalCode']:
    Latitude = ds.loc[ds.PostalCode == postalcode, 'Latitude'].iloc[0]
    Longitude = ds.loc[ds.PostalCode == postalcode, 'Longitude'].iloc[0]
    neighborhoods.loc[neighborhoods.PostalCode == postalcode, 'Latitude'] = Latitude
    neighborhoods.loc[neighborhoods.PostalCode == postalcode, 'Longitude'] = Longitude
    
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
10,M3B,North York,Don Mills North,43.745906,-79.352188
11,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
13,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
